Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math as math

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  
  loss = tf.reduce_mean(
  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regu = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regu)
#   l2_loss = tf.nn.l2_loss(weights)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 51.366577
Training accuracy: 7.2%
Validation accuracy: 10.3%
Loss at step 100: 11.881418
Training accuracy: 73.8%
Validation accuracy: 73.0%
Loss at step 200: 4.485557
Training accuracy: 79.2%
Validation accuracy: 77.7%
Loss at step 300: 1.981242
Training accuracy: 82.4%
Validation accuracy: 80.5%
Loss at step 400: 1.130878
Training accuracy: 83.9%
Validation accuracy: 81.7%
Loss at step 500: 0.838613
Training accuracy: 84.2%
Validation accuracy: 82.3%
Loss at step 600: 0.736970
Training accuracy: 84.2%
Validation accuracy: 82.4%
Loss at step 700: 0.701293
Training accuracy: 84.3%
Validation accuracy: 82.5%
Loss at step 800: 0.688676
Training accuracy: 84.3%
Validation accuracy: 82.6%
Test accuracy: 88.7%


In [6]:
batch_size = 128
beta = 0.01
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  regu = tf.nn.l2_loss(weights)

  loss = tf.reduce_mean(loss + beta * regu)
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [7]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 45.607025
Minibatch accuracy: 14.1%
Validation accuracy: 11.6%
Minibatch loss at step 500: 0.861034
Minibatch accuracy: 79.7%
Validation accuracy: 82.0%
Minibatch loss at step 1000: 0.993989
Minibatch accuracy: 76.6%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 0.899460
Minibatch accuracy: 79.7%
Validation accuracy: 81.7%
Minibatch loss at step 2000: 0.972578
Minibatch accuracy: 76.6%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.829967
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 3000: 0.700535
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Minibatch loss at step 3500: 0.894999
Minibatch accuracy: 77.3%
Validation accuracy: 81.2%
Minibatch loss at step 4000: 0.717523
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 4500: 0.968045
Minibatch accuracy: 78.1%
Validation accuracy: 82.1%
Minibatch loss at step 5000: 0.736213
Minibatch accuracy: 84.4%
Validation accura

In [8]:
hidden_layer_size = 1024
# num_steps = 5001
def run_model(dataset, w1, b1, w2, b2):
    first_layer = tf.matmul(dataset, w1) + b1
    hidden_layer = tf.nn.relu(first_layer)
    logits = tf.matmul(hidden_layer, w2) + b2
    return logits

def train_model(num_relus=0, batch_size=128, num_steps=5001):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size * image_size])
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
        b1 = tf.Variable(tf.zeros([hidden_layer_size]))
        w2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
        b2 = tf.Variable(tf.zeros([num_labels]))
        
        logits = run_model(tf_train_dataset, w1, b1, w2, b2)
        regu = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        
        loss = tf.reduce_mean(loss + beta * regu)
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(run_model(valid_dataset, w1, b1, w2, b2))
        test_prediction = tf.nn.softmax(run_model(test_dataset, w1, b1, w2, b2))
        
    with tf.Session(graph=graph) as session:
        # Initialize variables
        tf.global_variables_initializer().run()
        print("Initialized!")
        for i in range(num_steps):
            offset = (batch_size * i) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size),:]
            batch_labels = train_labels[offset:(offset + batch_size)]
            feed_dict = {
                tf_train_dataset: batch_data,
                tf_train_labels: batch_labels,
            }
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if i % 500 == 0:
                print("===========================")
                print("Step: {}".format(i))
                print("Loss: {}".format(l))
                print("Training Accuracy: {}".format(accuracy(predictions, batch_labels)))
                print("Validation Accuracy: {}".format(accuracy(valid_prediction.eval(), valid_labels)))
        print("Test Accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

In [9]:
train_model()

Initialized!
Step: 0
Loss: 3505.16455078
Training Accuracy: 11.71875
Validation Accuracy: 35.68
Step: 500
Loss: 21.32097435
Training Accuracy: 82.03125
Validation Accuracy: 84.91
Step: 1000
Loss: 1.09004497528
Training Accuracy: 81.25
Validation Accuracy: 84.24
Step: 1500
Loss: 0.785159468651
Training Accuracy: 81.25
Validation Accuracy: 84.14
Step: 2000
Loss: 0.901152193546
Training Accuracy: 78.125
Validation Accuracy: 83.96
Step: 2500
Loss: 0.756244182587
Training Accuracy: 82.8125
Validation Accuracy: 83.87
Step: 3000
Loss: 0.707436680794
Training Accuracy: 84.375
Validation Accuracy: 84.01
Step: 3500
Loss: 0.799469351768
Training Accuracy: 79.6875
Validation Accuracy: 83.8
Step: 4000
Loss: 0.671920955181
Training Accuracy: 83.59375
Validation Accuracy: 84.53
Step: 4500
Loss: 0.924611210823
Training Accuracy: 75.78125
Validation Accuracy: 84.2
Step: 5000
Loss: 0.699045658112
Training Accuracy: 85.15625
Validation Accuracy: 83.87
Test Accuracy: 89.22


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [10]:
train_dataset = train_dataset[:500,:]
train_labels = train_labels[:500]
hidden_layer_size = 1024
# num_steps = 5001
def run_model(dataset, w1, b1, w2, b2):
    first_layer = tf.matmul(dataset, w1) + b1
    hidden_layer = tf.nn.relu(first_layer)
    logits = tf.matmul(hidden_layer, w2) + b2
    return logits

def train_model(num_relus=0, batch_size=128, num_steps=3001):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size * image_size])
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
        b1 = tf.Variable(tf.zeros([hidden_layer_size]))
        w2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
        b2 = tf.Variable(tf.zeros([num_labels]))
        
        logits = run_model(tf_train_dataset, w1, b1, w2, b2)
        regu = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        
        loss = tf.reduce_mean(loss + beta * regu)
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(run_model(valid_dataset, w1, b1, w2, b2))
        test_prediction = tf.nn.softmax(run_model(test_dataset, w1, b1, w2, b2))
        
    with tf.Session(graph=graph) as session:
        # Initialize variables
        tf.global_variables_initializer().run()
        print("Initialized!")
        for i in range(num_steps):
            offset = (batch_size * i) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size),:]
            batch_labels = train_labels[offset:(offset + batch_size),:]
            feed_dict = {
                tf_train_dataset: batch_data,
                tf_train_labels: batch_labels,
            }
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if i % 500 == 0:
                print("===========================")
                print("Step: {}".format(i))
                print("Loss: {}".format(l))
                print("Training Accuracy: {}".format(accuracy(predictions, batch_labels)))
                print("Validation Accuracy: {}".format(accuracy(valid_prediction.eval(), valid_labels)))
        print("Test Accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

In [11]:
train_model()

Initialized!
Step: 0
Loss: 3484.51245117
Training Accuracy: 11.71875
Validation Accuracy: 27.12
Step: 500
Loss: 21.0140209198
Training Accuracy: 100.0
Validation Accuracy: 76.97
Step: 1000
Loss: 0.450817286968
Training Accuracy: 100.0
Validation Accuracy: 77.86
Step: 1500
Loss: 0.279822528362
Training Accuracy: 100.0
Validation Accuracy: 77.78
Step: 2000
Loss: 0.26774713397
Training Accuracy: 100.0
Validation Accuracy: 77.88
Step: 2500
Loss: 0.255785226822
Training Accuracy: 100.0
Validation Accuracy: 77.92
Step: 3000
Loss: 0.257705539465
Training Accuracy: 100.0
Validation Accuracy: 77.8
Test Accuracy: 83.7


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
train_dataset = train_dataset[:500,:]
train_labels = train_labels[:500]
hidden_layer_size = 1024
# num_steps = 5001
def run_model_train(dataset, w1, b1, w2, b2, keep_prob):
    first_layer = tf.matmul(dataset, w1) + b1
    hidden_layer = tf.nn.relu(first_layer)
    activations = tf.nn.dropout(hidden_layer, keep_prob)
    logits = tf.matmul(activations, w2) + b2
    return logits

def run_model(dataset, w1, b1, w2, b2):
    first_layer = tf.matmul(dataset, w1) + b1
    hidden_layer = tf.nn.relu(first_layer)
    logits = tf.matmul(hidden_layer, w2) + b2
    return logits

def train_model(num_relus=0, batch_size=128, num_steps=3001):
    graph = tf.Graph()
    with graph.as_default():
        tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size * image_size])
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size]))
        b1 = tf.Variable(tf.zeros([hidden_layer_size]))
        w2 = tf.Variable(tf.truncated_normal([hidden_layer_size, num_labels]))
        b2 = tf.Variable(tf.zeros([num_labels]))
        keep_prob = tf.placeholder("float")
        logits = run_model_train(tf_train_dataset, w1, b1, w2, b2, keep_prob)
        regu = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        
        loss = tf.reduce_mean(loss + beta * regu)
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(run_model(valid_dataset, w1, b1, w2, b2))
        test_prediction = tf.nn.softmax(run_model(test_dataset, w1, b1, w2, b2))
        
    with tf.Session(graph=graph) as session:
        # Initialize variables
        tf.global_variables_initializer().run()
        print("Initialized!")
        for i in range(num_steps):
            offset = (batch_size * i) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size),:]
            batch_labels = train_labels[offset:(offset + batch_size),:]
            feed_dict = {
                tf_train_dataset: batch_data,
                tf_train_labels: batch_labels,
                keep_prob: 0.5
            }
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if i % 500 == 0:
                print("===========================")
                print("Step: {}".format(i))
                print("Loss: {}".format(l))
                print("Training Accuracy: {}".format(accuracy(predictions, batch_labels)))
                print("Validation Accuracy: {}".format(accuracy(valid_prediction.eval(), valid_labels)))
        print("Test Accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

In [13]:
train_model()

Initialized!
Step: 0
Loss: 3651.09521484
Training Accuracy: 10.9375
Validation Accuracy: 34.85
Step: 500
Loss: 21.0834255219
Training Accuracy: 100.0
Validation Accuracy: 77.56
Step: 1000
Loss: 0.48790538311
Training Accuracy: 100.0
Validation Accuracy: 77.98
Step: 1500
Loss: 0.316199481487
Training Accuracy: 100.0
Validation Accuracy: 77.99
Step: 2000
Loss: 0.30302888155
Training Accuracy: 100.0
Validation Accuracy: 78.15
Step: 2500
Loss: 0.277829706669
Training Accuracy: 100.0
Validation Accuracy: 77.99
Step: 3000
Loss: 0.288499176502
Training Accuracy: 100.0
Validation Accuracy: 77.93
Test Accuracy: 83.75


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [14]:
hidden_layer_size = 1024
hidden_layer_size2 = 512
hidden_layer_size3 = 256
hidden_layer_size4 = 128
hidden_layer_size5 = 64
beta = 0.0001
# num_steps = 5001
def run_model_train(dataset, w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6, keep_prob):
    first_layer = tf.matmul(dataset, w1) + b1
    hidden_layer = tf.nn.relu(first_layer)
    activations = tf.nn.dropout(hidden_layer, keep_prob)
    
    second_layer = tf.matmul(activations, w2) + b2
    hidden_layer2 = tf.nn.relu(second_layer)
    activations2 = tf.nn.dropout(hidden_layer2, keep_prob)
    
    third_layer = tf.matmul(activations2, w3) + b3
    hidden_layer3 = tf.nn.relu(third_layer)
    activations3 = tf.nn.dropout(hidden_layer3, keep_prob)
    
    fourth_layer = tf.matmul(activations3, w4) + b4
    hidden_layer4 = tf.nn.relu(fourth_layer)
    activations4 = tf.nn.dropout(hidden_layer4, keep_prob)
    
    fifth_layer = tf.matmul(activations4, w5) + b5
    hidden_layer5 = tf.nn.relu(fifth_layer)
    activations5 = tf.nn.dropout(hidden_layer5, keep_prob)
    
    logits = tf.matmul(activations5, w6) + b6
    return logits

def run_model(dataset, w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6):
    first_layer = tf.matmul(dataset, w1) + b1
    hidden_layer = tf.nn.relu(first_layer)
    
    second_layer = tf.matmul(hidden_layer, w2) + b2
    hidden_layer2 = tf.nn.relu(second_layer)
    
    third_layer = tf.matmul(hidden_layer2, w3) + b3
    hidden_layer3 = tf.nn.relu(third_layer)
    
    fourth_layer = tf.matmul(hidden_layer3, w4) + b4
    hidden_layer4 = tf.nn.relu(fourth_layer)
    
    fifth_layer = tf.matmul(hidden_layer4, w5) + b5
    hidden_layer5 = tf.nn.relu(fifth_layer)
    
    logits = tf.matmul(hidden_layer5, w6) + b6
    return logits


def train_model(num_relus=0, batch_size=128, num_steps=30001):
    graph = tf.Graph()
    with graph.as_default():
        ## Datasets and variable Initializations
        tf_train_dataset = tf.placeholder(tf.float32, shape=[batch_size, image_size * image_size])
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        w1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_layer_size],
                                             stddev=math.sqrt(2.0/(image_size*image_size))))
        b1 = tf.Variable(tf.zeros([hidden_layer_size]))
        
        w2 = tf.Variable(tf.truncated_normal([hidden_layer_size, hidden_layer_size2],
                                             stddev=math.sqrt(2.0/hidden_layer_size)))
        b2 = tf.Variable(tf.zeros([hidden_layer_size2]))
        
        w3 = tf.Variable(tf.truncated_normal([hidden_layer_size2, hidden_layer_size3],
                                             stddev=math.sqrt(2.0/hidden_layer_size2)))
        b3 = tf.Variable(tf.zeros([hidden_layer_size3]))
        
        w4 = tf.Variable(tf.truncated_normal([hidden_layer_size3, hidden_layer_size4],
                                             stddev=math.sqrt(2.0/hidden_layer_size3)))
        b4 = tf.Variable(tf.zeros([hidden_layer_size4]))
        
        w5 = tf.Variable(tf.truncated_normal([hidden_layer_size4, hidden_layer_size5],
                                             stddev=math.sqrt(2.0/hidden_layer_size4)))
        b5 = tf.Variable(tf.zeros([hidden_layer_size5]))
        
        w6 = tf.Variable(tf.truncated_normal([hidden_layer_size5, num_labels],
                                             stddev=math.sqrt(2.0/hidden_layer_size5)))
        b6 = tf.Variable(tf.zeros([num_labels]))
        
        
        keep_prob = tf.placeholder("float")
        logits = run_model_train(tf_train_dataset, w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6, keep_prob)
        regu = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + tf.nn.l2_loss(w4) + tf.nn.l2_loss(w5) + tf.nn.l2_loss(w6)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
        
        loss = tf.reduce_mean(loss + beta * regu)
        global_step = tf.Variable(0)  # count the number of steps taken.
        learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
        
        train_prediction = tf.nn.softmax(logits)
        valid_prediction = tf.nn.softmax(run_model(valid_dataset, w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6))
        test_prediction = tf.nn.softmax(run_model(test_dataset, w1, b1, w2, b2, w3, b3, w4, b4, w5, b5, w6, b6))
        
    with tf.Session(graph=graph) as session:
        # Initialize variables
        tf.global_variables_initializer().run()
        print("Initialized!")
        for i in range(num_steps):
            offset = (batch_size * i) % (train_labels.shape[0] - batch_size)
            batch_data = train_dataset[offset:(offset + batch_size),:]
            batch_labels = train_labels[offset:(offset + batch_size),:]
            feed_dict = {
                tf_train_dataset: batch_data,
                tf_train_labels: batch_labels,
                keep_prob: 0.5
            }
            _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
            if i % 500 == 0:
                print("===========================")
                print("Step: {}".format(i))
                print("Loss: {}".format(l))
                print("Training Accuracy: {}".format(accuracy(predictions, batch_labels)))
                print("Validation Accuracy: {}".format(accuracy(valid_prediction.eval(), valid_labels)))
        print("Test Accuracy: {}".format(accuracy(test_prediction.eval(), test_labels)))

In [15]:
train_model()

Initialized!
Step: 0
Loss: 3.29892277718
Training Accuracy: 11.71875
Validation Accuracy: 12.06
Step: 500
Loss: 1.033816576
Training Accuracy: 85.9375
Validation Accuracy: 68.95
Step: 1000
Loss: nan
Training Accuracy: 5.46875
Validation Accuracy: 10.0
Step: 1500
Loss: nan
Training Accuracy: 4.6875
Validation Accuracy: 10.0
Step: 2000
Loss: nan
Training Accuracy: 4.6875
Validation Accuracy: 10.0
Step: 2500
Loss: nan
Training Accuracy: 4.6875
Validation Accuracy: 10.0
Step: 3000
Loss: nan
Training Accuracy: 6.25
Validation Accuracy: 10.0
Step: 3500
Loss: nan
Training Accuracy: 4.6875
Validation Accuracy: 10.0
Step: 4000
Loss: nan
Training Accuracy: 7.03125
Validation Accuracy: 10.0
Step: 4500
Loss: nan
Training Accuracy: 7.8125
Validation Accuracy: 10.0
Step: 5000
Loss: nan
Training Accuracy: 8.59375
Validation Accuracy: 10.0
Step: 5500
Loss: nan
Training Accuracy: 10.15625
Validation Accuracy: 10.0
Step: 6000
Loss: nan
Training Accuracy: 9.375
Validation Accuracy: 10.0
Step: 6500
Loss: 